# gazeNet 

## load pretrained model and configuration


In [40]:
import os
from utils_lib import utils

dev = False
model_dir = 'model_final'
model_name = 'gazeNET_0004_00003750'

logdir =  os.path.join('logdir', model_dir)
fname_config = os.path.join(logdir, 'config.json')
configuration = utils.Config(fname_config)
config = configuration.params

config['split_seqs']=False
config['augment']=False
config['batch_size']=1

cuda = False
model_name = '%s.pth.tar'%model_name

## load model

In [41]:
from model import gazeNET as gazeNET
import model as model_func
num_classes = len(config['events'])
model = gazeNET(config, num_classes)
model_func.load(model, model_dir, config, model_name)
model.eval()

logdir/model_final/models/gazeNET_0004_00003750.pth.tar
Loading model: logdir/model_final/models/gazeNET_0004_00003750.pth.tar
done.


gazeNET(
  (conv_stack): Sequential(
    (conv1): Sequential(
      (0): Conv2d(1, 8, kernel_size=(2, 11), stride=(1, 1), padding=(1, 5), bias=False)
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=0, max_val=20, inplace=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (conv2): Sequential(
      (0): Conv2d(8, 8, kernel_size=(2, 11), stride=(1, 1), padding=(1, 5), bias=False)
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=0, max_val=20, inplace=True)
      (3): Dropout(p=0.25, inplace=False)
    )
  )
  (rnn_stack): Sequential(
    (gru1): BatchRNN(
      (rnn): GRU(32, 64, bias=False, batch_first=True, bidirectional=True)
      (batch_norm_op): SequenceWise (
      BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
      (dropout_op): Dropout(p=0.25, inplace=False)
    )
    (gru2): BatchRNN(
      (rnn): GR

## Data upload

In [42]:
import pandas as pd
import numpy as np
import ipywidgets as widgets

uploader = widgets.FileUpload(
    accept='.tsv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)
display(uploader)

print('Upload complete!')
# if uploader.data 

FileUpload(value={}, accept='.tsv', description='Upload', multiple=True)

Upload complete!


In [43]:

kwargs = {
    'cuda': False,
    'use_tqdm': False,
    'eval': False,
}
root = 'etdata'
dataset = 'tobii'
fname = 'Kreuze_Random Recording1_short'
format = 'tsv'

fpath = '%s/%s/%s.%s'%(root, dataset, fname, format)
num_workers = 0

fdir, fname = os.path.split(os.path.splitext(fpath)[0])


## Input trial setup

Enter measurements for Your eye-tracking-setup.

The default values for monitor width and height are for a 24" 16:9 monitor.

In [44]:
width_mon = widgets.BoundedFloatText(
    value=53.1,
    min=10,
    max=100,
    step=0.1,
    description='Monitor size in cm:',
    disabled=False
)
display(width_mon)

height_mon = widgets.BoundedFloatText(
    value=29.8,
    min=10,
    max=100,
    step=0.1,
    description='Monitor size in cm:',
    disabled=False
)
display(height_mon)

res_hor = widgets.BoundedIntText(
    value=1920,
    min=640,
    max=3840,
    step=10,
    description='Horizontal resolution:',
    disabled=False
)
display(res_hor)

res_vert = widgets.BoundedIntText(
    value=1080,
    min=480,
    max=2160,
    step=10,
    description='Vertical resolution:',
    disabled=False
)
display(res_vert)

geom = {
    'screen_width': height_mon.value,
    'screen_height': width_mon.value,
    'display_width_pix': res_hor.value,
    'display_height_pix': res_vert.value,
    'eye_distance': 60,
}

BoundedFloatText(value=53.1, description='Monitor size in cm:', min=10.0, step=0.1)

BoundedFloatText(value=29.8, description='Monitor size in cm:', min=10.0, step=0.1)

BoundedIntText(value=1920, description='Horizontal resolution:', max=3840, min=640, step=10)

BoundedIntText(value=1080, description='Vertical resolution:', max=2160, min=480, step=10)

## load data

In [45]:
from utils_lib.data_loader import EMDataset, GazeDataLoader
from utils_lib.etdata import tsv_to_npy

if format == 'tsv':
   x, y, X_test = tsv_to_npy(fpath, geom)
else:
    X_test = np.load(fpath)

_status = np.isnan(X_test['x']) | \
        np.isnan(X_test['y']) | \
        ~np.in1d(X_test['evt'], config['events'])
X_test['status'] = ~_status
test_dataset = EMDataset(config = config, gaze_data = [X_test])
n_samples = len(test_dataset)
test_loader = GazeDataLoader(test_dataset, batch_size=config['batch_size'],
                            num_workers=num_workers,
                            shuffle=False)

## label data

In [46]:
from utils_lib.ETeval import run_infer

print ("Predicting %s" % fname)
# _gt, _pr, pr_raw = run_infer(model, n_samples, test_loader, **kwargs)

Predicting Kreuze_Random Recording1_short


## postprocessing

In [47]:
# from utils_lib.etdata import ETData
# import copy


# # revert to kartesian
# if format == 'tsv':
#     X_test['x'] = x
#     X_test['y'] = y

# #glue back the predictions
# _data_pr = copy.deepcopy(test_dataset.data)
# for _d, _pred in zip(_data_pr, pr_raw):
#     _d['evt'] = 0
#     _d['evt'][1:] = np.argmax(_pred, axis=1)+1
# _data_pr = pd.concat([pd.DataFrame(_d) for _d in _data_pr]).reset_index(drop=True)
# _data = pd.DataFrame(X_test)
# _data = _data.merge(_data_pr, on='t', suffixes=('', '_pred'), how='left')
# _data['evt'] = _data['evt_pred'].replace({np.nan:0})

# etdata_pr = ETData()
# etdata_pr.load(_data[['t', 'x', 'y', 'status', 'evt']].values, **{'source':'np_array'})

# sdir = fdir.replace(dataset, '%s_gazeNet_copy'%dataset)
# if not os.path.exists(sdir):
#     os.mkdir(sdir)
# spath = '%s/%s'%(sdir, fname)
# etdata_pr.save(spath)
# etdata_pr.plot_px(show=True, save=False, spath='%s'%spath)


In [48]:
# etdata_pr.plot_xy(show=True, save=False, spath='%s'%spath)

## Download files

In [49]:
import solara

f = open(fpath, "rb")
solara.FileDownload(data=f, label="Download image", filename="image.jpeg")
widgets.VBox([
    solara.FileDownload.widget(data=f, label="Download image", filename=fpath)
])
display(solara.FileDownload.widget(data=f, label="Download image", filename=fpath))


Container(value=None, children=(FileDownloadWidget(children=[Btn(children=[Icon(children=['mdi-cloud-download-…